In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

/home/mcb/users/rdeng4/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get the raw datas
train_dataloader = DataLoader(training_data, batch_size=5000, shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=500, shuffle=True)

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [4]:
# obtain img_raw data and one-hot key encoded labels
torch.manual_seed(2022)
train_imgs, train_labels = next(iter(train_dataloader))

In [5]:
# define the model, only a MLP

class MyNN(nn.Module):
    def __init__(self, img_shape, unit_dim=512):
        super(MyNN,self).__init__()
        self.img_shape = img_shape
        self.unit_dim = unit_dim
    
        self.linear = nn.Sequential(
            nn.Flatten(), # 28x28 -> 784, can include flatten layer into the sequential layer
            nn.Linear(img_shape[0]*img_shape[1],unit_dim), # 784 -> 512
            nn.ReLU(),
            # nn.MaxPool1d(stride=5,padding=3,kernel_size=2),
            nn.Linear(unit_dim,10), # 512 -> 10
            nn.Softmax(dim=1) # change this shape 10 from digit form to prob form (softmax)
        )
        


    def forward(self, img):
        # img = self.flatten(img)
        pred = self.linear(img)
        return pred

In [6]:
try_img = train_imgs[0]
try_label = train_labels[0]
img_shape = try_img.squeeze().shape
try_img = try_img.to(device) # put the data used to train the model to the same device as the model

# print(try_img[0].shape)
model = MyNN(img_shape).to(device)
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters(): 
    # print the parameters (the weight matrixes created for each layer)
    # since there is no weight matrix associated with activation function, there will be no params for activation layer
    # however, for the linear
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} | Shape: {param.shape}\n")

Model structure: MyNN(
  (linear): Sequential(
    (0): Flatten()
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=10, bias=True)
    (4): Softmax(dim=1)
  )
)


Layer: linear.1.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0114,  0.0262, -0.0106,  ..., -0.0127, -0.0006, -0.0354],
        [ 0.0337, -0.0190,  0.0019,  ..., -0.0236, -0.0147,  0.0114]],
       device='cuda:0', grad_fn=<SliceBackward>) | Shape: torch.Size([512, 784])

Layer: linear.1.bias | Size: torch.Size([512]) | Values : tensor([-0.0068, -0.0236], device='cuda:0', grad_fn=<SliceBackward>) | Shape: torch.Size([512])

Layer: linear.3.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0396, -0.0366,  0.0029,  ..., -0.0102, -0.0164,  0.0336],
        [-0.0259, -0.0366,  0.0430,  ...,  0.0364,  0.0066, -0.0140]],
       device='cuda:0', grad_fn=<SliceBackward>) | Shape: torch.Size([10, 512])

Layer: linear.3.bias | Size: torch

In [7]:
pred = model(try_img)
label = torch.argmax(pred)
true_label = torch.argmax(try_label)
print(f"Predicted class: {label}, True class: {true_label}")

Predicted class: 8, True class: 4


# Actual Training With CNN